---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd
import re 

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)
        
df = pd.Series(doc)

In [3]:
def date_sorter():

    import numpy as np
    import pandas as pd
    from datetime import date
    
    # ex1: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    ex1 = r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})'   
    DF1 = df.str.extractall(ex1)
    DF1.columns = ["month", "day", "year"]
    DF1 = DF1.reset_index()

    # ex2: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    # ex2: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    ex2= r'(?:(\d{1,2})\s)?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*(?:-|\.|,|\s)\s?(\d{,2}[a-z]*)(?:-|\.|\s|,|\.)?\s?(\d{4})'
    DF2 = df.str.extractall(ex2)
    DF2.columns = ["day", "month","Day", "year"]
    DF2 = DF2.reset_index()  
    DF2['day'] = DF2['day'].fillna(DF2['Day'])
    del DF2['Day']
    DF2['day'] = DF2['day'].fillna('1')
    DF2['day'] = DF2['day'].str.lstrip('0')
    DF2['month'] = DF2['month'].str.lstrip('0')
    DF2['month'] = DF2['month'].replace({"Jan":1, "Feb":2, "Mar":3, "Apr":4, "May":5, "Jun":6, "Jul":7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12 })

    ###

    ex3 = r'((\d{1,2})[/-](\d{2,4}))'
    DF3 = df.str.extractall(ex3)
    DF3 = DF3.reset_index()
    DF3 = DF3[132:]
    del DF3[0]
    DF3.columns = ['level_0' , 'match', 'month', 'year']
    DF3['month'] = DF3.month.str.lstrip('0') ##Removing 0's at the begining. 
    DF3['day']= '1' ## Filling day with 1. 
    DF3['index']= DF3['level_0']
    DF3 = DF3[['index', 'level_0', 'match', 'day', 'month', 'year']]
    DF3 = DF3.set_index('index')

    ###


    ex4 = r'(\d{4})'
    DF4 = df.str.extractall(ex4)
    DF4 = DF4.reset_index()
    DF4.columns= ['level_0', 'match', 'year']
    DF4 = DF4[403:]
    DF4.insert(0, column='day', value=np.nan)
    DF4.insert(1, column='month', value=np.nan)
    DF4['month']= DF4['month'].fillna('1')
    DF4['day']= DF4['day'].fillna('1')
    DF4['index']= DF4['level_0']
    DF4 = DF4[['index','level_0', 'match', 'day', 'month', 'year']]
    DF4 = DF4.set_index('index')



    result = DF1.append(DF2[~DF2.level_0.isin(DF1.level_0)])

    result = result.append(DF3[~DF3.level_0.isin(result.level_0)])

    result = result.append(DF4[~DF4.level_0.isin(result.level_0)])

    result = result[result.match!=1]  

    result['month'].fillna(1, inplace=True)
    result['day'].fillna(1, inplace=True)


    result = pd.DataFrame(result,columns = ["level_0", "match", "day", "month","year"])

    year1 = result['year'].str.len()==2
    year2 = result['year'].astype(int)>20

    result['year']=np.select([year1 & year2, year1 & ~year2], ['19','20'], '')+result['year']

    result.day.astype(int)
    result.month.astype(int)
    result.year.astype(int)
    result['date']= pd.to_datetime(result[["year", "month", "day"]])


    result.reset_index()
    result= result.sort_values(["date", "level_0"]).reset_index(drop=True)
    return result.level_0



In [4]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64

In [6]:
def date_sorter():

    import numpy as np
    import pandas as pd
    from datetime import date
    
    # ex1: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    ex1 = r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})'   
    DF1 = df.str.extractall(ex1)
    DF1.columns = ["month", "day", "year"]
    DF1 = DF1.reset_index()

    # ex2: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    # ex2: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    ex2= r'(?:(\d{1,2})\s)?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*(?:-|\.|,|\s)\s?(\d{,2}[a-z]*)(?:-|\.|\s|,|\.)?\s?(\d{4})'
    DF2 = df.str.extractall(ex2)
    DF2.columns = ["day", "month","Day", "year"]
    DF2 = DF2.reset_index()  
    DF2['day'] = DF2['day'].fillna(DF2['Day'])
    del DF2['Day']
    DF2['day'] = DF2['day'].fillna('1')
    DF2['day'] = DF2['day'].str.lstrip('0')
    DF2['month'] = DF2['month'].str.lstrip('0')
    DF2['month'] = DF2['month'].replace({"Jan":1, "Feb":2, "Mar":3, "Apr":4, "May":5, "Jun":6, "Jul":7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12 })

    ###

    ex3 = r'((\d{1,2})[/-](\d{2,4}))'
    DF3 = df.str.extractall(ex3)
    DF3 = DF3.reset_index()
    DF3 = DF3[132:]
    del DF3[0]
    DF3.columns = ['level_0' , 'match', 'month', 'year']
    DF3['month'] = DF3.month.str.lstrip('0') ##Removing 0's at the begining. 
    DF3['day']= '1' ## Filling day with 1. 
    DF3['index']= DF3['level_0']
    DF3 = DF3[['index', 'level_0', 'match', 'day', 'month', 'year']]
    DF3 = DF3.set_index('index')

    ###


    ex4 = r'(\d{4})'
    DF4 = df.str.extractall(ex4)
    DF4 = DF4.reset_index()
    DF4.columns= ['level_0', 'match', 'year']
    DF4 = DF4[403:]
    DF4.insert(0, column='day', value=np.nan)
    DF4.insert(1, column='month', value=np.nan)
    DF4['month']= DF4['month'].fillna('1')
    DF4['day']= DF4['day'].fillna('1')
    DF4['index']= DF4['level_0']
    DF4 = DF4[['index','level_0', 'match', 'day', 'month', 'year']]
    DF4 = DF4.set_index('index')



    result = DF1.append(DF2[~DF2.level_0.isin(DF1.level_0)])

    result = result.append(DF3[~DF3.level_0.isin(result.level_0)])

    result = result.append(DF4[~DF4.level_0.isin(result.level_0)])

    result = result[result.match!=1]  

    result['month'].fillna(1, inplace=True)
    result['day'].fillna(1, inplace=True)


    result = pd.DataFrame(result,columns = ["level_0", "match", "day", "month","year"])

    year1 = result['year'].str.len()==2
    year2 = result['year'].astype(int)>20

    result['year']=np.select([year1 & year2, year1 & ~year2], ['19','20'], '')+result['year']

    result.day.astype(int)
    result.month.astype(int)
    result.year.astype(int)
    result['date']= pd.to_datetime(result[["year", "month", "day"]])


    result.reset_index()
    result= result.sort_values(["date", "level_0"]).reset_index(drop=True)
    return result

date_sorter()



,level_0,match,day,month,year,date
0,9,0,10,4,1971,1971-04-10
1,84,0,18,5,1971,1971-05-18
2,2,0,8,7,1971,1971-07-08
3,53,0,11,7,1971,1971-07-11
4,28,0,12,9,1971,1971-09-12
5,474,0,1,1,1972,1972-01-01
6,153,0,13,1,1972,1972-01-13
7,13,0,26,1,1972,1972-01-26
8,129,0,6,5,1972,1972-05-06
9,98,0,13,5,1972,1972-05-13
